In [13]:
import pandas as pd
import numpy as np
import math
import os
import matplotlib.pyplot as plt
from scipy import optimize, stats
from itertools import chain

%matplotlib inline

In [31]:
### Загрузка данных интересующей нас пробы (полученные из MaxQuant, без доп обработки)
# Работаем с функциями для удобства в итерациях на несколько проб

# Путь к данным пробы

def initial(path):      

# Загрузка пробы

    data = pd.read_excel('Data/'+ path + '.xlsx') 
    df = pd.DataFrame(data)
    pat_prot=df['Majority protein IDs'].str.split(';')
    iBAQ_cols = [col for col in df.columns if 'iBAQ' in col]
    dt=[]
    headers=[]

# Оставляем только Accession № в названии белка

    for i in range(0,len(pat_prot)):
        s=pat_prot[i]
        if len(s)==1:
            try:
                pat_prot[i]=s[0].split('|')[1]
            except:
                pat_prot[i]=s[0]

    dt.append(pat_prot)
    headers.append('Majority protein IDs')

# Выбираем только колонки iBAQ из сырых данных

    for i in range(1,len(iBAQ_cols)):
        dt.append(df[iBAQ_cols[i]])
        headers.append(iBAQ_cols[i])

# Создание массива данных
    
    pat = pd. concat(dt, axis=1, keys=headers)
    
    return pat, headers

In [32]:
### Функция для подсчета коэф корреляции и коэффициентов калибровочной кривой. Будет использована в пересчете
# 'pat_' - проба, 'ref_' - референсные значения

def ib_quan(pat_,ref_,ref_er): 
    i=len(pat_)-1
    while i>=0:
        if pat_[i]==0 or pd.isna(pat_[i])==True:
            pat_=np.delete(pat_,i)
            ref_=np.delete(ref_,i)
            ref_er=np.delete(ref_er,i)
        i=i-1
        
    D=np.polyfit(np.log10(pat_),np.log10(ref_),1,w = [1.0/np.log10(ty) for ty in ref_er])
    r = np.corrcoef(np.log10(pat_),np.log10(ref_))

    return r ,D

In [9]:
### Получение коэффициентов для пересчета из 'iBAQ' в 'Copy number per platelet'

# Нахождение в нашей пробе белков из референсной таблички, если не нашли, то всместо индекса пишется '-1'

def coef(pat, headers,name,ref):

    a, c_ibaq, r, Da, Db,recalib = [],[],[],[],[],[]

    
    for i in range(0,len(ref.iloc[:,0])):
        pl=pat.index[pat['Majority protein IDs']==ref.iloc[:,0][i]].tolist()
        if len(pl)>0:
            a.append(pat.index[pat['Majority protein IDs']==ref.iloc[:,0][i]].tolist()[0])
        else:
            a.append(-1)

    yn = input("FAT (yes/no): ")
    
    if yn.lower() != 'yes':
            rec_val = [100,100,100,100]
    
# Создание массива, где белки упорядочены так же, как и в референсных данных
# Нахождение коэффициентов пересчета с помощью функции из предыдущего блока

    for j in range(0,len(headers)-1):

        if yn.lower() == 'yes':
            print(headers[j+1])
            f1, f2, f3 = input("Enter FAT data for CD42b, CD61, FSC: ").split()
            rec_val = [float(f1),float(f2),float(f3),float(f3)]

            
        c_ibaq.append([])
        r.append([])
        Da.append([])
        Db.append([])
        recalib.append([])
        for i in a:
            if i>=0:
                c_ibaq[j].append(pat.iloc[:,j+1][i])
            else:
                c_ibaq[j].append(0)            
        res=ib_quan(c_ibaq[j],ref.iloc[:,1].tolist(),ref.iloc[:,2].tolist())
        r[j].append(res[0])
        Da[j].append(res[1][0])
        Db[j].append(res[1][1])
        
        n=0
        c=0
        m=0
        for k in ['P07359', 'P05106','P04406','Q9BQE3']:
            try:
                a1=pat.loc[pat['Majority protein IDs']==k, pat.columns[j+1]].item() 
                a2=ref.loc[ref['Accession number']==k, 'Concentration'].item()*rec_val[m]/100
                c1=np.log10(a2)/np.log10(np.power(a1,res[1][0])*np.power(10,res[1][1]))
                c=c+c1
                n=n+1
                m+=1
            except:
                m+=1
        if n==0:
            recalib[j].append(1)
        else:
            recalib[j].append(c/n)
            
    return c_ibaq, r, Da, Db, recalib

In [34]:
### Пересчет данных пробы в концентрации с использованием полученных коэффициентов пересчета

def recalc(pat, headers, Da, Db, reclb, name):
    pat_recalc = pat.copy()

    for j in range(0,len(headers)-1):
        pat_recalc[headers[j+1]] = pat_recalc[[headers[j+1]]].apply(lambda x: 
                                np.power(x,Da[j][0]*reclb[j][0])*np.power(10,float(Db[j][0]*reclb[j][0])))
        pat_recalc = pat_recalc.rename({pat_recalc.columns[j+1]: name+'__# per plt '+str(j+1)}, axis=1)
    
    return pat_recalc